# Example Data process

In [1]:
import torch
import pandas as pd
import numpy as np
import global_resources as gr
import os


device = gr.set_device()
print(f"Current training device: {device.capitalize()}.")

# GLOBAL VARIABLES
DTYPE = torch.float64
START_LEARNING_RATE = torch.tensor([0.01], device = device, dtype = DTYPE)


# Example Data process
# Read data
data_path = os.path.join(gr.default_dir, r'Data\breast-cancer-wisconsin.data')
df = gr.read_and_return_pd_df(data_path)

df.replace('?', np.nan, inplace = True)
df.dropna(inplace = True)
# df.replace('?', -99999, inplace = True)
df.drop(['id'], axis = 1, inplace = True)
df["bare_nuclei"] = df["bare_nuclei"].astype(np.int64)
df.dropna(inplace = True)

X = np.array(df.drop(['class'], axis = 1)).astype('float32')
y = np.array(df['class']).astype('float32')

y = np.where(y == 4, 1, np.where(y == 2, -1, y))

X_gpu = torch.tensor(X, dtype = DTYPE, device = device)
y_gpu = torch.tensor(y, dtype = DTYPE, device = device)

Current training device: Cuda.
Reading files from: D:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\breast-cancer-wisconsin.data


In [2]:
import SVM.SVC as svc

Current training device: Cuda.


# shuffle_tensor_row_wise
(
    X: torch.Tensor):

In [3]:
print(X)

[[ 5.  1.  1. ...  3.  1.  1.]
 [ 5.  4.  4. ...  3.  2.  1.]
 [ 3.  1.  1. ...  3.  1.  1.]
 ...
 [ 5. 10. 10. ...  8. 10.  2.]
 [ 4.  8.  6. ... 10.  6.  1.]
 [ 4.  8.  8. ... 10.  4.  1.]]


In [4]:
rtn_value = svc.shuffle_tensor_row_wise(X_gpu)
print(rtn_value)
print(X)

tensor([[ 9.,  8.,  8.,  ...,  4.,  1.,  1.],
        [10.,  6.,  5.,  ...,  8.,  6.,  1.],
        [10.,  7.,  7.,  ...,  5.,  7.,  2.],
        ...,
        [ 6., 10., 10.,  ...,  7., 10.,  7.],
        [ 4.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 4.,  3.,  2.,  ...,  2.,  1.,  1.]], device='cuda:0',
       dtype=torch.float64)
[[ 5.  1.  1. ...  3.  1.  1.]
 [ 5.  4.  4. ...  3.  2.  1.]
 [ 3.  1.  1. ...  3.  1.  1.]
 ...
 [ 5. 10. 10. ...  8. 10.  2.]
 [ 4.  8.  6. ... 10.  6.  1.]
 [ 4.  8.  8. ... 10.  4.  1.]]


# create_random_weights_bias
(
    X: torch.Tensor, 
    ) -> torch.Tensor:

In [5]:
weights_bias = svc.create_random_weights_bias(X_gpu)
print(weights_bias)

Creating random weights and bias with dtype: torch.float64
tensor([0.1669, 0.2015, 0.1487, 0.7193, 0.6959, 0.4649, 0.5759, 0.2324, 0.5004,
        0.0000], device='cuda:0', dtype=torch.float64)


# cal_distances
(
    X: torch.Tensor, 
    weights_and_bias: torch.Tensor,
    ) -> torch.Tensor:

In [6]:
distances = svc.cal_distances(X_gpu, weights_bias)
print(distances.dtype)
print(distances.shape)

torch.float64
torch.Size([683])


# hinge_loss
(
    distances: torch.Tensor, 
    labels: torch.Tensor, 
    margin: float = 1.0
    ) -> torch.Tensor:

In [7]:
losses = svc.hinge_loss(distances, y_gpu, margin = 1.0)
print(losses)

tensor([4.5871], device='cuda:0', dtype=torch.float64)


# hinge_loss_l2_panalty
(
    distances: torch.Tensor, 
    weights_and_bias: torch.Tensor,
    labels: torch.Tensor, 
    margin: float = 1.0, 
    l2_reg: float = 1e-4
    ) -> torch.Tensor:

In [8]:
losses_l2 = svc.hinge_loss_l2_panalty(distances, weights_bias, y_gpu)
print(losses_l2)

tensor([4.5872], device='cuda:0', dtype=torch.float64)


# update_model
(
    X: torch.Tensor, 
    y: torch.Tensor, 
    weights_and_bias: torch.Tensor,
    learning_rate: float = 0.01, 
    l2_penalty: bool = False,
    ) -> tuple[float, torch.Tensor, torch.Tensor]:

In [9]:
weights_bias.requires_grad_(True)
loss, weights, bias = svc.update_model(X_gpu, y_gpu, weights_bias, START_LEARNING_RATE)
print(loss, weights, bias)

4.587146625289337 tensor([0.1477, 0.1930, 0.1395, 0.7106, 0.6822, 0.4561, 0.5623, 0.2242, 0.4935],
       device='cuda:0', dtype=torch.float64, grad_fn=<AsStridedBackward0>) tensor(-0.0065, device='cuda:0', dtype=torch.float64,
       grad_fn=<AsStridedBackward0>)


# train
(X: torch.Tensor, 
          y: torch.Tensor, 
          weights_and_bias: torch.Tensor, 
          num_epochs = NUM_EPOCHS, 
          start_learning_rate = LEARNING_RATE, 
          l2_penalty = False, 
          print_every = int(100), 
          delta_loss_breaker = DELTA_LOSS_BREAKER, 
          patience = 10,
          norm_output_or_not = False
          ) -> tuple[torch.Tensor, torch.Tensor]:

## Parameters preparing

In [10]:
X_gpu = torch.tensor(X, dtype = DTYPE, device = device)
y_gpu = torch.tensor(y, dtype = DTYPE, device = device)
weights_bias = svc.create_random_weights_bias(X = X_gpu)
lr_cpu = 0.01
START_LEARNING_RATE = torch.tensor([lr_cpu], device = device, dtype = DTYPE)
DELTA_LOSS_BREAKER = 5e-6
NUM_EPOCHS = int(1e4)

Creating random weights and bias with dtype: torch.float64


In [11]:
weights, bias = svc.train(
    X = X_gpu, 
    y = y_gpu, 
    num_epochs = NUM_EPOCHS, 
    delta_loss_breaker = DELTA_LOSS_BREAKER, 
    l2_penalty = True, 
    weights_and_bias = weights_bias, 
    start_learning_rate = START_LEARNING_RATE)

Training with loss function: hinge loss with l2 penalty on weights.
Epoch 0 | Loss: 5.270883269687018 | Delta loss: None
Epoch 1 | Loss: 5.14988443825158 | Delta loss: 0.12099883143543799
Epoch 100 | Loss: 0.2127744397484874 | Delta loss: 0.002209514164338844
Epoch 200 | Loss: 0.12524006869906212 | Delta loss: -0.0002106559275796005
Epoch 300 | Loss: 0.11079344647227297 | Delta loss: 6.077774369755984e-05
Epoch 400 | Loss: 0.10542738016373712 | Delta loss: 2.6666240926967455e-05
Epoch 500 | Loss: 0.10155913621684243 | Delta loss: 3.3721017262802744e-05
Epoch 600 | Loss: 0.09826588331668171 | Delta loss: 2.3736964052389897e-05
Delta Loss smaller than threshold: Epoch 621 | Loss: 0.0976112904655026 | Delta loss: 4.930756890744736e-06
Epoch 700 | Loss: 0.09553241190043979 | Delta loss: 4.091123690043441e-05
Delta Loss smaller than threshold: Epoch 704 | Loss: 0.09546661736099354 | Delta loss: 2.1293815433420793e-06
Epoch 800 | Loss: 0.09345203210285793 | Delta loss: 2.1564420762884562e-05

In [12]:
SVC = svc.extract_svc(weights, bias)
unnormed = svc.extract_svc(weights, bias, norm = False)

print(unnormed)
print(SVC)

tensor([ 0.1031,  0.0558,  0.1318,  0.0719,  0.0319,  0.1687,  0.0642,  0.0706,
         0.1238, -2.7276], device='cuda:0', dtype=torch.float64,
       grad_fn=<CatBackward0>)
tensor([ 0.3433,  0.1856,  0.4390,  0.2395,  0.1062,  0.5618,  0.2138,  0.2350,
         0.4121, -9.0819], device='cuda:0', dtype=torch.float64,
       grad_fn=<DivBackward0>)


In [13]:
accuracy = svc.score(weights, bias, X_gpu, y_gpu)
print(accuracy)

97.36456808199121
